# Bounding Box testing

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf

In [2]:
train_signs = np.load("lib/datasets/train_signs.npy")
test_signs = np.load("lib/datasets/test_signs.npy")
val_signs = np.load("lib/datasets/val_signs.npy")
train_labels_e = np.load("lib/datasets/train_labels_e.npy")
test_labels_e = np.load("lib/datasets/test_labels_e.npy")
val_labels_e = np.load("lib/datasets/val_labels_e.npy")
train_bboxes = np.load("lib/datasets/train_bboxes.npy")
test_bboxes = np.load("lib/datasets/test_bboxes.npy")
val_bboxes = np.load("lib/datasets/val_bboxes.npy")

In [5]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, ZeroPadding2D, BatchNormalization, Input, InputLayer

In [13]:
# input_1 (InputLayer)        [(None, 224, 224, 3)]     0 
model=Sequential()

model.add(Input(shape=train_signs.shape[1:]))
model.add(Conv2D(96,(11,11),strides=4,activation='relu',padding='valid'))
model.add(MaxPool2D((3,3),strides=2))
model.add(BatchNormalization())

model.add(ZeroPadding2D(padding=2))
model.add(Conv2D(256,(5,5),activation='relu',padding='same'))
model.add(MaxPool2D((3,3), strides=2))
model.add(BatchNormalization())

model.add(ZeroPadding2D(padding=1))
model.add(Conv2D(384,(3,3),activation='relu',padding='same'))

model.add(ZeroPadding2D(padding=1))
model.add(Conv2D(384,(3,3),activation='relu',padding='same'))

model.add(ZeroPadding2D(padding=1))
model.add(Conv2D(256,(3,3),activation='relu',padding='same'))

model.add(Flatten())
model.add(Dense(4096,activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(4096,activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(47,activation="softmax"))

model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.optimizers.SGD(learning_rate=0.001),
              metrics=['accuracy'])
#callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.1, patience=5, verbose=1,
    mode='max', min_delta=0.0001, cooldown=2, min_lr=0)
epochs = 50
history = model.fit(train_signs, train_labels_e, batch_size=32, epochs=epochs,
validation_data=(val_signs, val_labels_e), callbacks = [callback])

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

In [7]:
model.save("App/lib/models/AlexNet_bb")

INFO:tensorflow:Assets written to: App/lib/models/AlexNet_bb\assets


In [18]:
model = tf.keras.models.load_model('App/lib/models/AlexNet_bb')

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 zero_padding2d (ZeroPadding  (None, 30, 30, 96)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 256)       614656    
                                                      

In [9]:
for i in range(5):
    model.pop()

In [10]:
for layer in model.layers:
    layer.trainable = False

In [12]:
bboxHead = Sequential()
bboxHead.add(Dense(128, activation="relu"))
bboxHead.add(Dense(64, activation="relu"))
bboxHead.add(Dense(32, activation="relu"))
bboxHead.add(Dense(4, activation="sigmoid",name="bounding_box"))
softmaxHead = Sequential()
softmaxHead.add(Dense(512, activation="relu"))
softmaxHead.add(Dropout(0.5))
softmaxHead.add(Dense(512, activation="relu"))
softmaxHead.add(Dropout(0.5))
softmaxHead.add(Dense(47, activation="softmax",name="class_label"))
model_bb = Model(inputs=model,outputs=(bboxHead, softmaxHead))

ValueError: Found unexpected instance while processing input tensors for keras functional model. Expecting KerasTensor which is from tf.keras.Input() or output from keras layer call(). Got: <keras.engine.sequential.Sequential object at 0x00000136B7A32AC8>

In [7]:
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(4, activation="sigmoid"))

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 zero_padding2d (ZeroPadding  (None, 30, 30, 96)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 256)       614656    
                                                      

In [9]:
for layer in model.layers[:14]:
    layer.trainable = False

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 zero_padding2d (ZeroPadding  (None, 30, 30, 96)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 256)       614656    
                                                      

In [14]:
losses = {"class_label": "categorical_crossentropy","bounding_box": "mean_squared_error"}
model.compile(loss=losses, optimizer="adam")
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 zero_padding2d (ZeroPadding  (None, 30, 30, 96)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 256)       614656    
                                                      

In [16]:
train_targets = {"class_label": train_labels_e,"bounding_box": train_bboxes}
test_targets = {"class_label": test_labels_e,"bounding_box": val_bboxes}

In [17]:
H = model.fit(
	train_signs, train_targets,
	validation_data=(test_signs, test_targets),
	batch_size=32,
	epochs=30,
	verbose=1)

Epoch 1/30


ValueError: in user code:

    File "C:\Users\dokee\anaconda3\envs\tensor\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dokee\anaconda3\envs\tensor\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dokee\anaconda3\envs\tensor\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\dokee\anaconda3\envs\tensor\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\dokee\anaconda3\envs\tensor\lib\site-packages\keras\engine\training.py", line 919, in compute_loss
        y, y_pred, sample_weight, regularization_losses=self.losses)
    File "C:\Users\dokee\anaconda3\envs\tensor\lib\site-packages\keras\engine\compi

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 zero_padding2d (ZeroPadding  (None, 30, 30, 96)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 256)       614656    
                                                      

In [43]:
model.layers[15].trainable = True

In [45]:
model.compile(loss="mse", optimizer="adam")
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 zero_padding2d (ZeroPadding  (None, 30, 30, 96)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 256)       614656    
                                                      

In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 zero_padding2d (ZeroPadding  (None, 30, 30, 96)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 256)       614656    
                                                      

In [33]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
vgg = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)

In [34]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     